<a href="https://colab.research.google.com/github/AbdulRehman2106/Python-Assignment-4/blob/main/Countdown_Timer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import time
import datetime
import os
import curses
import pyttsx3

def type_print(text, delay=0.01):
    for char in text:
        print(char, end="", flush=True)
        time.sleep(delay)

def type_animate(text, dot_count=3, delay=0.5):
    for _ in range(dot_count):
        print(text + "." * (_ + 1), end="\r", flush=True)
        time.sleep(delay)
    print(" " * (len(text) + dot_count), end="\r")

def show_splash_screen():
    os.system('cls' if os.name == 'nt' else 'clear')

    title = """

 __          __  _                             _______         _____                  _      _                          _______ _                         _
 \ \        / / | |                           |__   __|       / ____|                | |    | |                        |__   __(_)                       | |
  \ \  /\  / /__| | ___ ___  _ __ ___   ___      | | ___     | |     ___  _   _ _ __ | |_ __| | _____      ___ __         | |   _ _ __ ___   ___ _ __    | |
   \ \/  \/ / _ \ |/ __/ _ \| '_ ` _ \ / _ \     | |/ _ \    | |    / _ \| | | | '_ \| __/ _` |/ _ \ \ /\ / / '_ \        | |  | | '_ ` _ \ / _ \ '__|   | |
    \  /\  /  __/ | (_| (_) | | | | | |  __/     | | (_) |   | |___| (_) | |_| | | | | || (_| | (_) \ V  V /| | | |       | |  | | | | | | |  __/ |      |_|   _ _
     \/  \/ \___|_|\___\___/|_| |_| |_|\___|     |_|\___/     \_____\___/ \__,_|_| |_|\__\__,_|\___/ \_/\_/ |_| |_|       |_|  |_|_| |_| |_|\___|_|      (_)  (_|_)

                                                   ⏳ COUNTDOWN  TIMER ⏳

                                                   Created By Abdul Rehman
    """
    type_print(title, delay=0.002)
    type_animate("\n🔄 Loading")
    print("\n")

try:
    import winsound
    winsound_available = True
except ImportError:
    winsound_available = False

try:
    tts = pyttsx3.init()
    use_tts = True
except ImportError:
    use_tts = False


class CountdownTimer:
    def __init__(self):
        self.paused = False
        self.running = False
        self.total_seconds = 0
        self.sound_choice = "Beep"
        self.custom_message = "Time's up!"
        self.reset_flag = False
        self.exit_flag = False

    def beep(self):
        if os.name == 'nt' and winsound_available:
            winsound.Beep(1000, 500)
        else:
            print('\a')

    def speak(self, text):
        if use_tts:
            tts.say(text)
            tts.runAndWait()

    def typing_effect(self, text, stdscr, y, delay=0.05):
        for i, char in enumerate(text):
            stdscr.addstr(y, i, char)
            stdscr.refresh()
            time.sleep(delay)

    def start_timer(self):
        try:
            h = int(input("\nEnter Hours: "))
            m = int(input("\nEnter Minutes: "))
            s = int(input("\nEnter Seconds: "))
            self.total_seconds = h * 3600 + m * 60 + s
            self.original_seconds = self.total_seconds
            if self.total_seconds == 0:
                raise ValueError
        except ValueError:
            print("\n⚠️ Invalid Input. Please Enter Valid Hours, Minutes, And Seconds.")
            return

        self.running = True
        self.paused = False
        curses.wrapper(self.run_countdown)

    def run_countdown(self, stdscr):
        curses.curs_set(0)
        stdscr.nodelay(True)
        height, width = stdscr.getmaxyx()
        start_time = self.total_seconds

        if height < 12 or width < 40:
            stdscr.clear()
            stdscr.addstr(1, 1, "\n⚠️ Please Enlarge Your Terminal Window!")
            stdscr.refresh()
            time.sleep(3)
            return

        while self.total_seconds > 0 and not self.exit_flag:
            stdscr.clear()

            if self.reset_flag:
                self.total_seconds = start_time
                self.reset_flag = False

            try:
                key = stdscr.getch()
                if key == ord('p'):
                    self.paused = not self.paused
                elif key == ord('r'):
                    self.reset_flag = True
                elif key == ord('q'):
                    self.exit_flag = True
                    break
            except:
                pass

            if not self.paused:
                mins, secs = divmod(self.total_seconds, 60)
                hours, mins = divmod(mins, 60)
                time_str = f"{hours:02}:{mins:02}:{secs:02}"
                bar_length = width - 20
                progress = int((start_time - self.total_seconds) / max(start_time, 1) * bar_length)
                bar = '#' * progress + '-' * (bar_length - progress)

                stdscr.addstr(1, 2, " \nAdvanced Countdown Timer")
                stdscr.addstr(3, 2, f"\nTime Remaining: {time_str}")
                stdscr.addstr(5, 2, f"\n[{bar[:bar_length]})")
                stdscr.addstr(7, 2, "\nControls: (P)ause / (R)eset / (Q)uit")

                clock = datetime.datetime.now().strftime("%H:%M:%S")
                stdscr.addstr(9, 2, f"\nCurrent Time: {clock}")
            else:
                stdscr.addstr(11, 2, "\nPaused. Press 'P' To Resume.")

            stdscr.refresh()
            time.sleep(1 if not self.paused else 0.1)
            if not self.paused:
                self.total_seconds -= 1

        if not self.exit_flag:
            self.beep()
            self.speak(self.custom_message)
            stdscr.clear()
            self.typing_effect("\nTIME'S UP! \nGreat job!", stdscr, height // 2)
            stdscr.refresh()
            time.sleep(3)
            self.log_session()

    def log_session(self):
        now = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        duration = str(datetime.timedelta(seconds=self.original_seconds - self.total_seconds))
        log_message = f"\n[{now}] Timer Finished With Message: {self.custom_message} | Duration: {duration}\n"
        with open("timer_log.txt", "a", encoding="utf-8") as log_file:
            log_file.write(log_message)

    def set_custom_message(self):
        custom_message = input("\nEnter Your Custom Alarm Message: ")
        if custom_message:
            self.custom_message = custom_message
            print(f"\nCustom Message Set To: {self.custom_message}")


if __name__ == "__main__":
    show_splash_screen()
    timer = CountdownTimer()

    while True:
        print("\nChoose an option:")
        print("\n1. Start Timer")
        print("\n2. Set Custom Message")
        print("\n3. Exit")

        choice = input("\nEnter Your Choice: ")

        if choice == '1':
            timer.start_timer()
        elif choice == '2':
            timer.set_custom_message()
        elif choice == '3':
            print("\nExiting Program...")
            break
        else:
            print("\n⚠️ Invalid Choice. Please Try Again.")
